In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import modules
import pickle

In [7]:
cross_entropy_mutation = nn.CrossEntropyLoss(
            weight=torch.tensor([0.1, 1.0, 1.0])
        )
t1 = torch.randn((1200, 3))
q = torch.randint(0, 3, (1200,))
loss = cross_entropy_mutation(t1, q)

### test loading data

In [1]:
import json
import os
import trainUtils

/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/data2/tyfei/trainresults/evomodel/train_siqi" 
with open(os.path.join(path, "config.json"), "r") as f:
    configs = json.load(f)
ds = trainUtils.loadDataset(configs)
pretrain_model = trainUtils.loadPretrainModel(configs)
print("build finetune model")
model = trainUtils.buildModel(configs, pretrain_model)

Seed set to 1509


build finetune model
model at stage: training stage 1


/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [4]:
model.training_step(list(d[129]), 0)

[ 0 20 18 ... 19 11  2]
[ 0 20 18 ... 19 11  2]


TypeError: 'int' object is not callable

In [3]:
d = ds.train_dataloader()
d = d.dataset
d[129]

get train loader


({'input': {'Spike': array([ 0, 20, 18, ..., 19, 11,  2]),
   'aligned_Spike': array([ 0, 20, 18, ..., 19, 11,  2])},
  'ori_seq': {'Spike': array([ 0, 20, 18, ..., 19, 11,  2])},
  'stage_1_masks': {'Spike': array([1, 1, 1, ..., 1, 1, 1], dtype=int32)},
  'label': {'Spike': []},
  'mutation_label': {'Spike': tensor([0, 0, 0,  ..., 0, 0, 0])},
  'meta': {'id': 'OEAV22454769', 'bin': '1610', 'days': 1615}},
 {'input': {'Spike': array([ 0, 20, 18, ..., 19, 11,  2]),
   'aligned_Spike': array([ 0, 20, 18, ..., 19, 11,  2])},
  'ori_seq': {'Spike': array([ 0, 20, 18, ..., 19, 11,  2])},
  'stage_1_masks': {'Spike': array([1, 1, 1, ..., 1, 1, 1], dtype=int32)},
  'label': {'Spike': []},
  'mutation_label': {'Spike': tensor([0, 0, 0,  ..., 0, 0, 0])},
  'meta': {'id': 'OEAV22474316', 'bin': '1631', 'days': 1636}})

In [24]:
d[129][0]["input"]

{'Spike': array([ 0, 20, 18, ..., 19, 11,  2]),
 'aligned_Spike': array([ 0, 20, 18, ..., 19, 11,  2])}

In [7]:
with open("/data/tyfei/datasets/covid/ori_seq3.pkl", "rb") as f:
    d = pickle.load(f)
spike = d["Spike"]
spike

array([ 0, 20, 18, ..., 19, 11,  2])

In [8]:
class testModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(30, 100) 
        self.decoder = modules.DecoderBlock(100, 5, 30, 3) 
    def forward(self, x):
        x = self.embed(x)
        x = self.decoder(x)
        return x

In [10]:
model = modules.DecoderBlock(1024, 16, 33, 5) 
optim = torch.optim.Adam(model.parameters(), lr=0.001)
cri = nn.CrossEntropyLoss() 
for i in range(100):
    x = torch.ones((1, 1275, 1024)).float()
    out = model(x)
    loss = cri(out.view(-1, 33), torch.tensor(spike).long())
    optim.zero_grad()
    loss.backward()
    optim.step()
    if i % 10 == 0:
        print(i, loss.item())
        # print(out.shape, spike.shape)
        # print(out[0, 0, :10], spike[:10])

0 3.7031874656677246
10 0.11883002519607544
20 0.0011939909309148788


KeyboardInterrupt: 

In [ ]:
x = torch.ones((1, 1275, 1024)).float()
out = model(x)
out = out.squeeze(0)
out = out.argmax(dim=-1)     
out

tensor([ 0, 20, 18,  ..., 19, 11,  2])

In [9]:
model = testModule()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
cri = nn.CrossEntropyLoss()
for i in range(200):
    x = torch.randint(1, 30, (32, 10))
    x[:, -1] = 0
    y = x.clone()
    y[:, 1:] = x[:, :-1]
    y[:, 0] = 0  # Set the first token to a specific value (e.g., 0)
    optim.zero_grad()
    out = model(x)
    loss = cri(out.view(-1, 30), y.view(-1))
    loss.backward()
    optim.step()
    if i % 20 == 0:
        print(f"Step {i}, Loss: {loss.item()}")

Step 0, Loss: 3.5705902576446533
Step 20, Loss: 2.9788079261779785
Step 40, Loss: 2.4528794288635254
Step 60, Loss: 1.0887776613235474
Step 80, Loss: 0.01829402893781662
Step 100, Loss: 0.003664557356387377
Step 120, Loss: 0.0020231674425303936
Step 140, Loss: 0.0016921693459153175
Step 160, Loss: 0.0012823834549635649
Step 180, Loss: 0.0011464199051260948
